In [2]:
from typing import List
import sympy as sym
from spatialmath import SE3
from spatialmath.base import *
import numpy as np
from numpy import sin, cos
from roboticstoolbox import ETS as ET
import roboticstoolbox as rtb
import matplotlib.pyplot as plt
from spatialmath import SE3

np.set_printoptions(precision=4, suppress=True, sign=" ")

In [5]:
def fkins(q5,q6, unit:str='rad'):
    pitch2yaw = 0.0092
    if unit == 'deg':
        q5 = q5*np.pi/180
        q6 = q6*np.pi/180

    alpha = 90 * np.pi/180
    #fmt:off
    transf = [ \
    [-sin(q5)*sin(q6)*cos(alpha)+cos(q5)*cos(q6), -sin(q5)*cos(alpha)*cos(q6)-sin(q6)*cos(q5),sin(alpha)*sin(q5),pitch2yaw*cos(q5)],\
    [ sin(q5)*cos(q6)+sin(q6)*cos(alpha)*cos(q5),-sin(q5)*sin(q6)+cos(alpha)*cos(q5)*cos(q6),-sin(alpha)*cos(q5),pitch2yaw*sin(q5)],\
    [ sin(alpha)*sin(q6), sin(alpha)*cos(q6), 0 , cos(alpha)],\
    [0,0,0,1]
    ]
    #fmt:on
    return np.array(transf)

def create_robot():
    pitch2yaw = 0.0092
    E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() *  ET.tx(0.1) * ET.tz(0.1) 
    robot = rtb.ERobot(E, name="test")
    return robot

def rtb_fkins(q5,q6,unit:str='rad'):
    pitch2yaw = 0.0092
    E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() #*  ET.tx(0.1) * ET.tz(0.1)
    robot = rtb.ERobot(E, name="test")
    return robot.fkine([q5,q6],unit=unit) 

In [6]:
# My implementation
print("my implementation")
print(fkins(30,45,'deg'))
print("robot toolbox implementation")
robot_sol = rtb_fkins(30,45,'deg').data[0]
print(robot_sol)
print("Difference between implementations")
print(fkins(30,45,'deg')-robot_sol)

my implementation
[[ 0.6124 -0.6124  0.5     0.008 ]
 [ 0.3536 -0.3536 -0.866   0.0046]
 [ 0.7071  0.7071  0.      0.    ]
 [ 0.      0.      0.      1.    ]]
robot toolbox implementation
[[ 0.6124 -0.6124  0.5     0.008 ]
 [ 0.3536 -0.3536 -0.866   0.0046]
 [ 0.7071  0.7071  0.      0.    ]
 [ 0.      0.      0.      1.    ]]
Difference between implementations
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0.  0.]
 [ 0.  0.  0.  0.]]


In [7]:
robot_sol = rtb_fkins(30,45,'deg')
robot = create_robot()
s = robot.ikine_LM(robot_sol)
print(s)
q = s.q*180/np.pi
print(q)

IKsolution(q=array([ 0.5245,  0.7854]), success=False, reason='rejected-step limit 100 exceeded', iterations=108, residual=0.14141836592818435)
[ 30.0527  45.    ]


In [8]:
pitch2yaw = 0.0092
E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() *  ET.tx(0.1) * ET.tz(0.1) 
robot1 = rtb.ERobot(E, name="test")

end_effector = SE3(0.1, 0.0, 0.1)
E = ET.rz() * ET.tx(pitch2yaw) * ET.rx(90, "deg") * ET.rz() 
robot2 = rtb.ERobot(E, name="test2",tool=end_effector)

robot_sol1 = robot1.fkine([30, 30], "deg")
robot_sol2 = robot2.fkine([30, 30], "deg")

s1 = robot1.ikine_LM(robot_sol1)
q1 = s1.q*180/np.pi
s2 = robot2.ikine_LM(robot_sol2)
q2 = s2.q*180/np.pi

print(robot_sol1)
print(s1)
print(q1)
print(robot_sol2)
print(s2)
print(q2)

   0.75     -0.433     0.5       0.133     
   0.433    -0.25     -0.866    -0.0387    
   0.5       0.866     0         0.05      
   0         0         0         1         

IKsolution(q=array([ 0.5236,  0.5236]), success=True, reason=None, iterations=7, residual=4.393974521071345e-11)
[ 30.  30.]
   0.75     -0.433     0.5       0.133     
   0.433    -0.25     -0.866    -0.0387    
   0.5       0.866     0         0.05      
   0         0         0         1         

IKsolution(q=array([ 0.5236,  0.5236]), success=True, reason=None, iterations=7, residual=4.393974521071345e-11)
[ 30.  30.]


## Mathematical derivation

In [9]:
#Transformation 1
theta = sym.symbols('theta5')
t1 = trotz(theta)
t1_sym = sym.Matrix(list(t1))
t1_sym

Matrix([
[cos(theta5), -sin(theta5), 0, 0],
[sin(theta5),  cos(theta5), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

In [10]:
#Transformation 2
pitch2yaw = 0.092
pitch2yaw = sym.symbols('pitch2yaw')
t2 = transl(pitch2yaw,0,0)
t2_sym = sym.Matrix(list(t2))
t2_sym

Matrix([
[1, 0, 0, pitch2yaw],
[0, 1, 0,         0],
[0, 0, 1,         0],
[0, 0, 0,         1]])

In [11]:
eps = 1e-12
#Transformation 3
# theta = sym.symbols('alpha')
# t3 = trotx(theta)
t3 = trotx(np.pi/2,unit='rad')
t3[abs(t3) < eps] = 0
t3_sym = sym.Matrix(list(t3))
t3_sym

Matrix([
[1.0,   0,    0,   0],
[  0,   0, -1.0,   0],
[  0, 1.0,    0,   0],
[  0,   0,    0, 1.0]])

In [12]:
#Transformation 4
theta = sym.symbols('theta6')
t4 = trotz(theta)
t4_sym = sym.Matrix(list(t4))
t4_sym

Matrix([
[cos(theta6), -sin(theta6), 0, 0],
[sin(theta6),  cos(theta6), 0, 0],
[          0,            0, 1, 0],
[          0,            0, 0, 1]])

In [13]:
total = t1_sym*t2_sym*t3_sym*t4_sym
total


Matrix([
[1.0*cos(theta5)*cos(theta6), -1.0*sin(theta6)*cos(theta5),  1.0*sin(theta5), 1.0*pitch2yaw*cos(theta5)],
[1.0*sin(theta5)*cos(theta6), -1.0*sin(theta5)*sin(theta6), -1.0*cos(theta5), 1.0*pitch2yaw*sin(theta5)],
[            1.0*sin(theta6),              1.0*cos(theta6),                0,                         0],
[                          0,                            0,                0,                       1.0]])

In [14]:
sym.trigsimp(total)

Matrix([
[1.0*cos(theta5)*cos(theta6), -1.0*sin(theta6)*cos(theta5),  1.0*sin(theta5), 1.0*pitch2yaw*cos(theta5)],
[1.0*sin(theta5)*cos(theta6), -1.0*sin(theta5)*sin(theta6), -1.0*cos(theta5), 1.0*pitch2yaw*sin(theta5)],
[            1.0*sin(theta6),              1.0*cos(theta6),                0,                         0],
[                          0,                            0,                0,                       1.0]])

# Trascendental equation solutions 

In [15]:
import numpy as np

a = 5 
b = 6 
theta = 45*np.pi/180
theta = 0.966718

c = a*np.cos(theta) + b*np.sin(theta) 

t = np.arctan2(np.sqrt(a**2+b**2-c**2),c) 
s1 = np.arctan2(b,a) + t
s2 = np.arctan2(b,a) - t

print(f"c:         {c:8.6f}")
print(f"theta:     {theta:8.6f}")
print(f"solution1: {s1:8.6f}")
print(f"solution2: {s2:8.6f}")


c:         7.778175
theta:     0.966718
solution1: 0.966718
solution2: 0.785398


# Full robot

In [10]:
def create_robot(q5,q6):
    pitch2yaw = 0.0092
    E = ET.rx(-90,"deg")* ET.rz((-90+q5)*np.pi/180) * ET.tx(pitch2yaw) * ET.rx(-90, "deg") * ET.rz((-90+q6)*np.pi/180) 
    return E 

pose = create_robot(0,0)
print(pose.eval())
print(type(pose.eval().data[0]))
print(pose.eval().data[0])

   0         0         1         0         
   1         0         0         0         
   0         1         0         0.0092    
   0         0         0         1         

<class 'numpy.ndarray'>
[[-0.      0.      1.      0.    ]
 [ 1.     -0.      0.     -0.    ]
 [ 0.      1.     -0.      0.0092]
 [ 0.      0.      0.      1.    ]]


# Joint 4 calculation


In [27]:
from roboticstoolbox.robot.DHLink import PrismaticMDH, RevoluteMDH
from roboticstoolbox.robot import DHRobot 
from numpy import pi 
import numpy as np 
from spatialmath import SE3

#fmt:off 
robot2tracker_T= [[ -0.9131, -0.4064,  0.0317,  0.0358],
                  [ -0.0455,  0.0243, -0.9986, -0.0933],
                  [  0.4051, -0.9133, -0.0407,  0.9338],
                  [  0.0000,  0.0000,  0.0000,  1.0000 ]]
robot2tracker_T = np.array(robot2tracker_T)
print(robot2tracker_T)
# robot2tracker_T = SE3(T=robot2tracker_T)

deg2rad = lambda x:x*pi/180
rad2deg = lambda x:x*180/pi

ltool = 0.4162
joint_3_4 = DHRobot([RevoluteMDH(a=0.0, alpha=0.0, d=ltool, offset=0)], base=robot2tracker_T) 

print(joint_3_4)
print(type(joint_3_4))

target_T = joint_3_4.fkine([deg2rad(45)])
print(target_T)

sol = joint_3_4.ikine_LM(target_T)
print(sol)
print(f"Solution: {rad2deg(sol.q)}")

[[-0.9131 -0.4064  0.0317  0.0358]
 [-0.0455  0.0243 -0.9986 -0.0933]
 [ 0.4051 -0.9133 -0.0407  0.9338]
 [ 0.      0.      0.      1.    ]]
DHRobot: noname, 1 joints (R), dynamics, modified DH parameters
┏━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━┓
┃aⱼ₋₁ ┃ ⍺ⱼ₋₁ ┃ θⱼ  ┃   dⱼ   ┃
┣━━━━━╋━━━━━━╋━━━━━╋━━━━━━━━┫
┃ 0.0 ┃ 0.0° ┃  q1 ┃ 0.4162 ┃
┗━━━━━┻━━━━━━┻━━━━━┻━━━━━━━━┛

<class 'roboticstoolbox.robot.DHRobot.DHRobot'>


ValueError: can't append different type of object